# Q-Learning

### Libraries

In [92]:
import gymnasium as gym
import numpy as np
import pygame as pg
import matplotlib.pyplot as plt
import imageio
from collections import defaultdict

### Env init

In [93]:
env = gym.make("FrozenLake-v1", render_mode="rgb_array", is_slippery=False)

### Variables and table

In [94]:
n_train_episodes = 10000
lr = 0.7
n_eval_episodes = 100
max_steps = 100
gamma = 0.95
min_epsilon = 0.05
max_epsilon = 1.0
decay = 0.0005

In [95]:
#'''
num_states = env.observation_space.n
num_actions = env.action_space.n
done = False
frames = []
Q_table = np.zeros((num_states, num_actions))

for ep in range(n_train_episodes):
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay * ep)
    obs, _ = env.reset()
    for _ in range(max_steps):
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = int(np.argmax(Q_table[obs]))
        new_obs, reward, terminated, truncated, _ = env.step(action)
        Q_table[obs, action] += lr * (reward + gamma * np.max(Q_table[new_obs]) - Q_table[obs, action])
        if terminated or truncated:
            break
        obs = new_obs

#'''

In [96]:
obs, _ = env.reset()

while True:
    action = np.argmax(Q_table[obs])
    new_obs, reward, terminated, truncated, _ = env.step(action)
    frames.append(env.render())
    if terminated or truncated:
        break
    obs = new_obs

env.close()
# Save the frames as a GIF
imageio.mimsave('results/frozenlake.gif', frames, fps=4)